In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', 200)

In [ ]:
# pip install tspytsl

In [ ]:
import pyTSL

c = pyTSL.Client("hit", 
                 "20230427",
                 "tsl.tinysoft.com.cn", 
                 443)
c.login() #成功返回 1，失败返回 0
r = c.exec('''return "测试"; ''')
if r.error(): #有错误
    print(r.message())
else:
    print(r.value())

In [ ]:
r = c.call("GetAbkbyDate","A股",dt.date(2020,12,7))
if r.error(): #有错误
    print(r.message())
else:
    print(pd.DataFrame(r.value()))

In [ ]:
def rdo(df,name):
  #本地路径+文件名（这里用代码）
    path="C:/Users/YiLab1/Desktop/"+name+".xlsx"
    df= pd.DataFrame(df)
    print(df.shape, path)
    return 1

pyTSL.register_proc("a",rdo)

test = '''
    function get_value(stocks,begt,endt);
    begin
     for i in stocks do
     begin
       stock:=stocks[i];
       t:= select ['StockID'],datetimetostr(['date']) as 'date',['close']
         from Markettable datekey inttodate(begt) to inttodate(endt) of stock end;
       rdo2 a(t,stock);
     end
     return array("执行完成");
    end;
    '''
stocks = ['SH600000','SH600004','SH600006','SH600007']

#获取开始日至截止日分钟线行情
r = c.call('get_value', stocks, 20210701,20210705 , code=test,cycle="1分钟线")
print(r.value())

In [ ]:
def write_file(df, name):
    df= pd.DataFrame(df)
    df.to_csv(name)
    return 1

pyTSL.register_proc("write_file",write_file)

func = '''
function get_data();
begin
    Dir:= '/Users/data/'; //文件存放路径
    begT:= 20230913T;
    endT:= 20230914T;
    QHArr:= getbk('上市期货;退市期货');
    Tarr:= MarketTradeDayQk(begT,endT); //市场交易日
    
    for i:=0 to length(Tarr)-1 do
    begin
        vEndt:=Tarr[i];
        //选出指定日在市交易的
        bk:= sselect thisrow from QHArr
                    where spec(FuturesIsTrade(vEndt),thisrow) =1 end; 
        //FileLJ:= datetostr(vEndt)$'/'; //按日期文件夹存放指定日数据
        for j:=0 to length(bk)-1 do
        begin
            setsysparam(pn_stock(),bk[j]);
            //if not istradeday(Tarr[i]) then continue;
            //取数
            //确定数据开始时间-考虑到期货有夜盘数据，则开始时间为上个交易日的收盘后
            with *,array(pn_cycle():cy_day(), pn_stock():'SH000001',pn_date():vEndt) do
            vBegT:=ref(sp_time(),1)+18/24;//一般18点前白盘能结束
            data:= select 
                datetimetostr(['date']) as 'time',
                ['StockID'] as 'symbol_id',
                ['StockName'] as 'symbol_name',
                ['price'] as 'last',
                ['vol'] as 'trade_volume',
                ['amount'] as 'trade_amount',
                ['cjbs'] as 'open_interest_change',
                ['bc1'] as 'bv1',
                ['buy1'] as 'bp1',
                ['sale1'] as 'ap1',
                ['sc1'] as 'av1',
                ['sectional_yclose'] as 'prev_close',
                ['sectional_open'] as 'open',
                ['sectional_high'] as 'high',
                ['sectional_low'] as 'low',
                ['sectional_vol'] as 'volume',
                ['sectional_amount'] as 'amount',
                ['sectional_cjbs'] as 'open_interest'
                    from tradetable datekey vBegT to vEndt+18/24 of bk[j] end;
            //导出 
            FileName:= bk[j]; //csv文件名-以证券代码进行命名 
            ret:= rdo2 write_file(data, FileName+'.csv');
            //打印执行进度及状态
            if ret then
                echo '导出'$FileLJ$'日'$FileName$'数据成功!'; 
            else 
                echo '导出'$FileLJ$'日'$FileName$'数据失败!';
        end
    end
    return '数据导出完毕!';
end
'''
r = c.call('get_data', code=func)